# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments           Customer  \
0  Jun  9 2022  9:08AM  239210        10  0085749584  ISDIN Corporation   
1  Jun  9 2022  9:08AM  239210        10  0085749582  ISDIN Corporation   
2  Jun  9 2022  9:08AM  239210        10  0085749583  ISDIN Corporation   
3  Jun  9 2022  9:08AM  239210        10  0085749598  ISDIN Corporation   
4  Jun  9 2022  9:08AM  239210        10  0085749601  ISDIN Corporation   
5  Jun  9 2022  9:08AM  239210        10  0085749599  ISDIN Corporation   
6  Jun  9 2022  9:08AM  239210        10  0085749600  ISDIN Corporation   
7  Jun  9 2022  9:08AM  239210        10  0085749652  ISDIN Corporation   
8  Jun  9 2022  9:08AM  239210        10  0085749655  ISDIN Corporation   
9  Jun  9 2022  9:08AM  239210        10  0085749657  ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
52  239210       Released         62
53  239211       Released          1
54  239212       Released          1
55  239213       Released          1
56  239214       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
239210                NaN        NaN      62.0
239211                NaN        NaN       1.0
239212                NaN        NaN       1.0
239213                NaN        NaN       1.0
239214                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
239075                8.0        3.0       1.0
239083                0.0        1.0       0.0
239086                0.0        0.0       1.0
239090                0.0        1.0       0.0
239107                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
239075                  8          3         1
239083                  0          1         0
239086                  0          0         1
239090                  0          1         0
239107                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               239075          8          3         1
1               239083          0          1         0
2               239086          0          0         1
3               239090          0          1         0
4               239107          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               239075         8         3        1
1               239083                   1         
2               239086                            1
3               239090                   1         
4               239107                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse  \
0    Jun  9 2022  9:08AM  239210        10   
62   Jun  9 2022  9:08AM  239213        19   
63   Jun  9 2022  9:07AM  239214        19   
64   Jun  9 2022  9:06AM  239212        19   
65   Jun  9 2022  9:05AM  239206        10   
87   Jun  9 2022  9:03AM  239211        10   
88   Jun  9 2022  8:59AM  239209        10   
93   Jun  9 2022  8:58AM  239208        10   
109  Jun  9 2022  8:51AM  239207        12   
110  Jun  9 2022  8:43AM  239205        19   

                                 Customer  
0                       ISDIN Corporation  
62                GUSA Granules USA, Inc.  
63   Bi-Coastal Pharma International, LLC  
64   Bi-Coastal Pharma International, LLC  
65                      ISDIN Corporation  
87                     Beach Products Inc  
88                      ISDIN Corporation  
93                      ISDIN Corporation  
109                             Hush Hush  
110                GCH Granules USA, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse  \
0  Jun  9 2022  9:08AM  239210        10   
1  Jun  9 2022  9:08AM  239213        19   
2  Jun  9 2022  9:07AM  239214        19   
3  Jun  9 2022  9:06AM  239212        19   
4  Jun  9 2022  9:05AM  239206        10   
5  Jun  9 2022  9:03AM  239211        10   
6  Jun  9 2022  8:59AM  239209        10   
7  Jun  9 2022  8:58AM  239208        10   
8  Jun  9 2022  8:51AM  239207        12   
9  Jun  9 2022  8:43AM  239205        19   

                               Customer Completed Executing Released  
0                     ISDIN Corporation                           62  
1               GUSA Granules USA, Inc.                            1  
2  Bi-Coastal Pharma International, LLC                            1  
3  Bi-Coastal Pharma International, LLC                            1  
4                     ISDIN Corporation                           22  
5                    Beach Products Inc                            1  
6                     ISDIN Corporation                            5  
7                     ISDIN Corporation                           16  
8                             Hush Hush                   1           
9                GCH Granules USA, Inc.                            2

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse  \
0  Jun  9 2022  9:08AM  239210        10   
1  Jun  9 2022  9:08AM  239213        19   
2  Jun  9 2022  9:07AM  239214        19   
3  Jun  9 2022  9:06AM  239212        19   
4  Jun  9 2022  9:05AM  239206        10   
5  Jun  9 2022  9:03AM  239211        10   
6  Jun  9 2022  8:59AM  239209        10   
7  Jun  9 2022  8:58AM  239208        10   
8  Jun  9 2022  8:51AM  239207        12   
9  Jun  9 2022  8:43AM  239205        19   

                               Customer Released Executing Completed  
0                     ISDIN Corporation       62                      
1               GUSA Granules USA, Inc.        1                      
2  Bi-Coastal Pharma International, LLC        1                      
3  Bi-Coastal Pharma International, LLC        1                      
4                     ISDIN Corporation       22                      
5                    Beach Products Inc        1                      
6                     ISDIN Corporation        5                      
7                     ISDIN Corporation       16                      
8                             Hush Hush                  1            
9                GCH Granules USA, Inc.        2

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse  \
0  Jun  9 2022  9:08AM  239210        10   
1  Jun  9 2022  9:08AM  239213        19   
2  Jun  9 2022  9:07AM  239214        19   
3  Jun  9 2022  9:06AM  239212        19   
4  Jun  9 2022  9:05AM  239206        10   

                               Customer Released Executing Completed  
0                     ISDIN Corporation       62                      
1               GUSA Granules USA, Inc.        1                      
2  Bi-Coastal Pharma International, LLC        1                      
3  Bi-Coastal Pharma International, LLC        1                      
4                     ISDIN Corporation       22

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse  \
0  Jun  9 2022  9:08AM  239210        10   
1  Jun  9 2022  9:08AM  239213        19   
2  Jun  9 2022  9:07AM  239214        19   
3  Jun  9 2022  9:06AM  239212        19   
4  Jun  9 2022  9:05AM  239206        10   

                               Customer  Released  Executing  Completed  
0                     ISDIN Corporation      62.0        NaN        NaN  
1               GUSA Granules USA, Inc.       1.0        NaN        NaN  
2  Bi-Coastal Pharma International, LLC       1.0        NaN        NaN  
3  Bi-Coastal Pharma International, LLC       1.0        NaN        NaN  
4                     ISDIN Corporation      22.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse  \
0  Jun  9 2022  9:08AM  239210        10   
1  Jun  9 2022  9:08AM  239213        19   
2  Jun  9 2022  9:07AM  239214        19   
3  Jun  9 2022  9:06AM  239212        19   
4  Jun  9 2022  9:05AM  239206        10   

                               Customer  Released  Executing  Completed  
0                     ISDIN Corporation      62.0        0.0        0.0  
1               GUSA Granules USA, Inc.       1.0        0.0        0.0  
2  Bi-Coastal Pharma International, LLC       1.0        0.0        0.0  
3  Bi-Coastal Pharma International, LLC       1.0        0.0        0.0  
4                     ISDIN Corporation      22.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3826857     183.0        1.0       26.0
102         478176       1.0        1.0        0.0
12          478290       0.0        2.0        0.0
16          956566       2.0        2.0        0.0
18          717485       3.0        0.0        0.0
19         2152670      39.0       25.0       28.0
20          956478      24.0       16.0       13.0
21          478313       2.0        0.0        0.0
22          239200       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3826857     183.0        1.0       26.0
1       102   478176       1.0        1.0        0.0
2        12   478290       0.0        2.0        0.0
3        16   956566       2.0        2.0        0.0
4        18   717485       3.0        0.0        0.0
5        19  2152670      39.0       25.0       28.0
6        20   956478      24.0       16.0       13.0
7        21   478313       2.0        0.0        0.0
8        22   239200       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     183.0        1.0       26.0
1       102       1.0        1.0        0.0
2        12       0.0        2.0        0.0
3        16       2.0        2.0        0.0
4        18       3.0        0.0        0.0
5        19      39.0       25.0       28.0
6        20      24.0       16.0       13.0
7        21       2.0        0.0        0.0
8        22       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  183.0
1       102  Released    1.0
2        12  Released    0.0
3        16  Released    2.0
4        18  Released    3.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10  102   12   16   18    19    20   21   22
Status                                                    
Completed   26.0  0.0  0.0  0.0  0.0  28.0  13.0  0.0  0.0
Executing    1.0  1.0  2.0  2.0  0.0  25.0  16.0  0.0  0.0
Released   183.0  1.0  0.0  2.0  3.0  39.0  24.0  2.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10  102   12   16   18    19    20   21   22
0          Completed   26.0  0.0  0.0  0.0  0.0  28.0  13.0  0.0  0.0
1          Executing    1.0  1.0  2.0  2.0  0.0  25.0  16.0  0.0  0.0
2           Released  183.0  1.0  0.0  2.0  3.0  39.0  24.0  2.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10  102   12   16   18    19    20   21   22
0  Completed   26.0  0.0  0.0  0.0  0.0  28.0  13.0  0.0  0.0
1  Executing    1.0  1.0  2.0  2.0  0.0  25.0  16.0  0.0  0.0
2   Released  183.0  1.0  0.0  2.0  3.0  39.0  24.0  2.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()